In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml

In [ ]:
import araalictl

In [ ]:
import api

In [ ]:
import imp
imp.reload(api)

In [ ]:
#run = api.Runtime()

In [ ]:
%%time
zone, app = "prod", "bendvm"
araalictl.get_links(zone, app)

In [ ]:
links = _

In [ ]:
[api.Link(b, zone, app) for b in links]